### DEinstagram crawling to csv
   
- daily_gamsung.csv 
- gamsung.bnb.csv 
- gamsung_curation.csv 
- hi.stay.tour.csv 
- rest_behappyhere.csv 
- sookso.diary.csv 
- sookso.hada.csv

1. 숙소이름 추출  
2. 계정별 숙소 리스트 취합  
3. 통합 숙소리스트를 기준으로 content 병합  
4. like 수 합치기  
5. content 일괄 전처리(토큰화, 불용어제거)  

### 최종 데이터 셋  
columns = 숙소명, content(통합), like(통합), place(api), tags(통합), 게시수(중복값)


In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import re

import time
import os
import sys
import urllib.request
import json

from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
from pyspark.sql.functions import array_contains, udf


spark = SparkSession\
        .builder\
        .appName('Python Spark SQL basic example')\
        .getOrCreate()

sc = spark.sparkContext

daily_gamsung_DFP = spark.read.parquet("hdfs://localhost:9000/data/daily_gamsung_DF") ### daily_gamsung(완료)
gamsung_bnb_DFP = spark.read.parquet("hdfs://localhost:9000/data/gamsung_bnb_DF") ### gamsun.bnb
gamsung_curation_DFP = spark.read.parquet("hdfs://localhost:9000/data/gamsung_curation_DF") ### gamsung_curation(완료)
gamsung_hi_stay_tour_DFP = spark.read.parquet("hdfs://localhost:9000/data/hi_stay_tour_DF")  ### hi.stay.tour.csv (완료)
rest_behappyhere_DFP = spark.read.parquet("hdfs://localhost:9000/data/rest_behappyhere_DF") ### rest_behappyhere.csv
sookso_diary_DFP = spark.read.parquet("hdfs://localhost:9000/data/sookso_diary_DF") ### sookso.diary(완료)
sookso_hada_DFP = spark.read.parquet("hdfs://localhost:9000/data/sookso_hada_DF") ### sookso.hada (완료)

def daily_gamsung_name_extract(x):
    p = re.compile('\"[가-힣\t\n\r\f\v\s\_가-힣]+\"')
    m = p.findall(x)
    return m[0][1:-1]

def gamsung_bnb_name_extract(x):
    p = re.compile("\'[가-힣]+\'") # 
    m = p.findall(x)
    try:
        return m[0][1:-1]
    except :
        return ''
def gamsung_curation_name_extract(x):
    p = re.compile('\📍[가-힣]+') # 
    m = p.findall(x)
    try:
        return m[0][1:] 
    except :
        return ''
    

def hi_stay_tour_name_extract(x):
    p = re.compile("\#[가-힣]+") # 
    m = p.findall(x)
    try:
        return m[0][1:] 
    except :
        return ''

def rest_behappyhere_name_extract(x):
    p = re.compile('\#[가-힣]+')
    m = p.findall(x)
    return m[1][1:]

def sookso_diary_name_extract(x):
    p = re.compile('\📍 [가-힣]+\s[가-힇]+|\📍[가-힣]+\s[가-힇]+')
    m = p.findall(x)
    try:
        return m[0].split(' ')[-1]
    except:
        return ''
    

def sookso_hada_name_extract(x):
    p = re.compile('\#[가-힣]+')
    m = p.findall(x)
    if  m[1] == '#숙소하다': m[1], m[0] = m[0], m[1]
    return m[1][1:]



daily_gamsung_name = daily_gamsung.content.apply(lambda x : extract(x))
daily_gamsung['name'] = daily_gamsung_name

# name data가 없는 index 지우기
indexNames = daily_gamsung[daily_gamsung['name'] == ''].index
daily_gamsung.drop(indexNames, inplace=True) 



gamsung_bnb_name = gamsung_bnb.content.apply(lambda x : extract(x)) # gamsung_bnb_name = 
gamsung_bnb['name'] = gamsung_bnb_name


# name data가 없는 index 지우기
indexNames = gamsung_bnb[gamsung_bnb['name'] == ''].index
gamsung_bnb.drop(indexNames, inplace=True) 



gamsung_curation_name = gamsung_curation.content.apply(lambda x : extract(x))
gamsung_curation['name'] = gamsung_curation_name

# name data가 없는 index 지우기
indexNames = gamsung_curation[gamsung_curation['name'] == ''].index
gamsung_curation.drop(indexNames, inplace=True)



hi_stay_tour_name = hi_stay_tour.content.apply(lambda x : extract(x))
hi_stay_tour['name'] = hi_stay_tour_name


# name data가 없는 index 지우기
indexNames = hi_stay_tour[hi_stay_tour['name'] == ''].index
hi_stay_tour.drop(indexNames, inplace=True) 

rest_behappyhere_name = rest_behappyhere.content.apply(lambda x : extract(x))
rest_behappyhere['name'] = rest_behappyhere_name


# name data가 없는 index 지우기
indexNames = rest_behappyhere[rest_behappyhere['name'] == ''].index
rest_behappyhere.drop(indexNames, inplace=True) 


sookso_diary_name = sookso_diary.content.apply(lambda x : extract(x))
sookso_diary['name'] = sookso_diary_name

# name data가 없는 index 지우기
indexNames = sookso_diary[sookso_diary['name'] == ''].index
sookso_diary.drop(indexNames, inplace=True) 


sookso_hada_name = sookso_hada.content.apply(lambda x : extract(x))
sookso_hada['name'] = sookso_hada_name

# name data가 없는 index 지우기
indexNames = sookso_hada[sookso_hada['name'] == ''].index
sookso_hada.drop(indexNames, inplace=True) 


# like integer 변환 후 합치기
def convert(x):
    return int(x.replace(',',''))

if daily_gamsung.like.dtype == 'O':
    daily_gamsung.like = daily_gamsung.like.apply(lambda x : convert(x))
    

# 1,2,3
dg_cnt = Counter(daily_gamsung.name)
li =[]
for name, cnt  in dg_cnt.items():
    if cnt > 1:
        li.append(name)      

# 4. 행가져와서 like집계, content, tag list append
for i in li:           # 중복된 name 반복
    ex = daily_gamsung.loc[daily_gamsung.name == i]  # 반복된 name의 행을 가져온다
    ex.reset_index(drop=True,inplace=True)     # index reset
    
    content = ex.iloc[0,:].content
    date = ex.iloc[0,:].date
    like = ex.iloc[0,:].like
    place = ex.iloc[0,:].place
    tags = ex.iloc[0,:].tags
    name = ex.name[0]
    
    for i in range(len(ex)-1):
        content += ex.iloc[i+1,:].content
        like += ex.iloc[i+1,:].like
        tags += ex.iloc[i+1,:].tags
    
    # 5.차례대로 기존 행 지워내기
    indexNames = daily_gamsung[daily_gamsung['name'] == ex.name[0]].index
    daily_gamsung.drop(indexNames, inplace=True) 
    daily_gamsung = daily_gamsung.append(pd.Series(data=[content,date,like,place,tags,name],index=daily_gamsung.columns),ignore_index=True)

dg_tmp = []
for i in daily_gamsung.name:
    dg_tmp.append(dg_cnt[i])
    
daily_gamsung['overlap'] = dg_tmp
daily_gamsung

In [516]:


# like integet 변환 후 합치기
def convert(x):
    return int(x.replace(',',''))

if gamsung_bnb.like.dtype == 'O':
    gamsung_bnb.like = gamsung_bnb.like.apply(lambda x : convert(x))
    
    

# 1, 중복된 이름 검색 
# 2.개수가 1인거 이상 뽑아내기 
# 3.리스트만들기
gb_cnt = Counter(gamsung_bnb.name)      # Counter 모듈로 name data count
li =[]
for name, cnt  in gb_cnt.items():            # 중복된 것만 뽑기 (cnt가 1이상이면 중복)
    if cnt > 1:
        li.append(name)

# 4. 행가져와서 like집계, content, tag list append
for i in li:           # 중복된 name 반복
    ex = gamsung_bnb.loc[gamsung_bnb.name == i]  # 반복된 name의 행을 가져온다
    ex.reset_index(drop=True,inplace=True)     # index reset
    
    content = ex.iloc[0,:].content
    date = ex.iloc[0,:].date
    like = ex.iloc[0,:].like
    place = ex.iloc[0,:].place
    tags = ex.iloc[0,:].tags
    name = ex.name[0]
    
    for i in range(len(ex)-1):
        content += ex.iloc[i+1,:].content
        like += ex.iloc[i+1,:].like
        tags += ex.iloc[i+1,:].tags
        
    
    
    # 5.차례대로 기존 행 지워내기
    indexNames = gamsung_bnb[gamsung_bnb['name'] == ex.name[0]].index
    gamsung_bnb.drop(indexNames, inplace=True) 
    gamsung_bnb = gamsung_bnb.append(pd.Series(data=[content,date,like,place,tags,name],index=gamsung_bnb.columns),ignore_index=True)

gb_tmp = []
for i in gamsung_bnb.name:
    gb_tmp.append(gb_cnt[i])


gamsung_bnb['overlap'] = gb_tmp
gamsung_bnb

,content,date,like,place,tags,name,overlap
0,#제주_앤디앤라라홈제주 애월에 위치하고있는 한적한 유럽 시골집같은 공간 '앤디앤라라...,2021-09-08,213,애월읍 소길리,"['#제주_앤디앤라라홈제주', '#감성비앤비🏷제주로', '#감성비앤비_제주사진출처:...",앤디앤라라홈,1
1,#서울_고운서울 종로 도심 속 한옥주거지인 북촌한옥마을 언덕 위에 위치해 탁 트인 ...,2021-09-07,337,"Seoul, Korea","['#서울_고운서울', '#감성비앤비🏷서울로', '#감성비앤비_서울사진출처:', '...",고운,1
2,#경주_달무리스테이경북 경주에 위치하고있는 아늑한 분위기의 신상 한옥독채숙소 '달무...,2021-09-06,961,경주 황리단길,"['#경주_달무리스테이경북', '#감성비앤비🏷경주로', '#감성비앤비_경주사진출처:...",달무리스테이,1
3,#고성_삼박한집강원도 고성에 위치하고있는 따뜻한 느낌의 공간 '삼박한집''삼박한집'...,2021-09-05,531,삼박한집,"['#고성_삼박한집강원도', '#감성비앤비🏷고성으로', '#감성비앤비_고성사진출처:...",삼박한집,1
4,#부산_로스트펭귄부산 광안리 주택가에 위치한 편안한 휴식처가 되어줄 '로스트펭귄'독...,2021-09-02,439,"Busan, South Korea","['#부산_로스트펭귄부산', '#감성비앤비🏷부산으로', '#감성비앤비_부산사진출처:...",로스트펭귄,1
...,...,...,...,...,...,...,...
171,#포항_포레메르풀빌라포항 구룡포에 위치한 동해바다를 담은 프라이빗 풀빌라숙소 '포레...,2021-09-04,1322,Pohang,"['#포항_포레메르풀빌라포항', '#감성비앤비🏷포항으로', '#감성비앤비_포항포레메...",포레메르풀빌라,2
172,#밀양_밀양로그경남 밀양 얼음골에 위치한 동화 속 아기자기한 분위기가 느껴지는 감성...,2021-08-29,2065,Miryang,"['#밀양_밀양로그경남', '#감성비앤비🏷밀양으로', '#감성비앤비_밀양사진출처:'...",밀양로그,2
173,#감성비앤비이벤트여러분들의 많은 관심 덕분에 4만 기념으로 오랜만에 돌아온 감성비앤...,2021-08-19,1713,남포동,['#감성비앤비이벤트여러분들의']['#감성비앤비이벤트여러분들의'],팔로우,2
174,"#경주_라온채경주 불국사 근처에 위치하고있는, 오직 한 팀만을 위한 공간, 숙소에서...",2021-08-11,1986,라온채,"['#경주_라온채경주', '#감성비앤비🏷경주로', '#감성비앤비_경주사진출처:', ...",라온채,2


In [522]:

'''
'b'       boolean
'i'       (signed) integer
'u'       unsigned integer
'f'       floating-point
'c'       complex-floating point
'O'       (Python) objects
'S', 'a'  (byte-)string
'U'       Unicode
'V'       raw data (void)
'''



def convert(x):
    return int(x.replace(',',''))

if gamsung_curation.like.dtype == 'O':
    gamsung_curation.like = gamsung_curation.like.apply(lambda x : convert(x))


gc_cnt = Counter(gamsung_curation.name)      # Counter 모듈로 name data count
li =[]
for name, cnt  in gc_cnt.items():            # 중복된 것만 뽑기 (cnt가 1이상이면 중복)
    if cnt > 1:
        li.append(name)        
        

# 4. 행가져와서 like집계, content, tag list append
for i in li:           # 중복된 name 반복
    ex = gamsung_curation.loc[gamsung_curation.name == i]  # 반복된 name의 행을 가져온다
    ex.reset_index(drop=True,inplace=True)     # index reset
    
    content = ex.iloc[0,:].content
    date = ex.iloc[0,:].date
    like = ex.iloc[0,:].like
    place = ex.iloc[0,:].place
    tags = ex.iloc[0,:].tags
    name = ex.name[0]
    
    for i in range(len(ex)-1):
        content += ex.iloc[i+1,:].content
        like += ex.iloc[i+1,:].like
        tags += ex.iloc[i+1,:].tags
    
    # 5.차례대로 기존 행 지워내기
    indexNames = gamsung_curation[gamsung_curation['name'] == ex.name[0]].index
    gamsung_curation.drop(indexNames, inplace=True) 
    gamsung_curation = gamsung_curation.append(pd.Series(data=[content,date,like,place,tags,name],index=gamsung_curation.columns),ignore_index=True)

gc_tmp = []
for i in gamsung_curation.name:
    gc_tmp.append(gc_cnt[i])

gamsung_curation['overlap'] = gc_tmp
gamsung_curation.tail(20)

,content,date,like,place,tags,name,overlap
235,.유럽 리조트인줄알았는데 영덕이래!!!!완전 고급스러운 인테리어에 오션뷰는 덤 ❤️...,2019-02-21,469,None,"['#감성숙소_경상북도✔', '#감성숙소_영덕...', '#경상북도', '#영덕군'...",하벳부띠끄리조트,1
236,.와.. 여기펜션 분위기 뭐야..?독채에다 장작까지 진심 오져따ㅠㅠㅠㅠㅠ게다가 서울...,2019-02-09,696,None,"['#감성숙소_양평...', '#양평', '#책속에풍덩-']",책속에풍덩,1
237,.아니.. 우리나라에 이런데가 있다구..?일본감성 그대로라는 양평의 료칸이래ㅠㅠㅠㅠ...,2019-02-01,644,None,"['#감성숙소_양평...', '#경기도', '#양평', '#길조호텔-']",길조호텔,1
238,✅ #홍천_올라운드원조용한 숲 속에 자리잡은 외국 갤러리 같은 둥근 공간 안에 모던...,2021-08-18,2834,올라운드원,"['#홍천_올라운드원조용한', '#감성숙소_홍천✔', '#감성숙소_강원도📍올라운드원...",올라운드원,2
239,"✅ #공주_소소아한옥고요하고 아늑한 한옥의 정취를 느낄 수 있는 ""소소아한옥"" 총 ...",2021-08-15,1372,충남 공주시,"['#공주_소소아한옥고요하고', '#감성숙소_공주✔', '#감성숙소_충청남도📍소소아...",소소아한옥,2
240,✅ #포항_마치블루풀빌라⠀푸르른 하늘과 동해바다의 청량함을 온종일 감상할 수 있는 ...,2021-08-11,3388,마치블루 풀빌라,"['#포항_마치블루풀빌라⠀푸르른', '#감성숙소_포항✔', '#감성숙소_경상북도📍마...",마치블루풀빌라,2
241,✅ #거제_디스커버리풀빌라⠀찬란하게 빛나는 에메랄드 바다를 바라보며 힐링하기 좋은 ...,2021-08-07,12380,거제도,"['#거제_디스커버리풀빌라⠀찬란하게', '#감성숙소_거제✔', '#감성숙소_경상남도...",디스커버리풀빌라,5
242,"✅ #봉화_홀리가든산 중턱에 위치한 평화로운 비나리마을 안에 조용히 자리잡은 ""홀리...",2021-08-02,2201,홀리가든,"['#봉화_홀리가든산', '#감성숙소_봉화✔', '#감성숙소_경상북도📍홀리가든📍경북...",홀리가든,2
243,"✅ #경주_아나톨리아..에메랄드빛 바다가 선물하는 청량함을 온전히 느낄 수 있는 ""...",2021-07-13,6271,경주,"['#경주_아나톨리아..에메랄드빛', '#감성숙소_경주✔', '#감성숙소_경상북도....",아나톨리아,3
244,✅ #여수_블루망고풀빌라..에메랄드 빛 여수 바다를 바라보며 해외 휴양지에 온 듯한...,2021-07-02,4875,여수 블루망고 풀빌라 앤 리조트,"['#여수_블루망고풀빌라..에메랄드', '#감성숙소_여수✔', '#감성숙소_전라남도...",여수블루망고풀빌라,2


In [526]:


def convert(x):
    return int(x.replace(',',''))

if hi_stay_tour.like.dtype == 'O':
    hi_stay_tour.like = hi_stay_tour.like.apply(lambda x : convert(x))


# 1, 중복된 이름 검색 
# 2.개수가 1인거 이상 뽑아내기 
# 3.리스트만들기
hst_cnt = Counter(hi_stay_tour.name)      # Counter 모듈로 name data count
li =[]
for name, cnt  in hst_cnt.items():            # 중복된 것만 뽑기 (cnt가 1이상이면 중복)
    if cnt > 1:
        li.append(name)  

# 4. 행가져와서 like집계, content, tag list append
for i in li:           # 중복된 name 반복
    ex = hi_stay_tour.loc[hi_stay_tour.name == i]  # 반복된 name의 행을 가져온다
    ex.reset_index(drop=True,inplace=True)     # index reset
    
    content = ex.iloc[0,:].content
    date = ex.iloc[0,:].date
    like = ex.iloc[0,:].like
    place = ex.iloc[0,:].place
    tags = ex.iloc[0,:].tags
    name = ex.name[0]
    
    for i in range(len(ex)-1):
        content += ex.iloc[i+1,:].content
        like += ex.iloc[i+1,:].like
        tags += ex.iloc[i+1,:].tags
        
    
    
    # 5.차례대로 기존 행 지워내기
    indexNames = hi_stay_tour[hi_stay_tour['name'] == ex.name[0]].index
    hi_stay_tour.drop(indexNames, inplace=True) 
    hi_stay_tour = hi_stay_tour.append(pd.Series(data=[content,date,like,place,tags,name],index=hi_stay_tour.columns),ignore_index=True)

hst_cnt_tmp = []
for i in hi_stay_tour.name:
    hst_cnt_tmp.append(hst_cnt[i])

hi_stay_tour['overlap'] = hst_cnt_tmp
hi_stay_tour

,content,date,like,place,tags,name,overlap
0,🏷 월산산과 충주호 사이 자연속 작은숙소 #월악산유스호텔- - - - - - - -...,2021-09-06,61,월악산유스호스텔,"['#월악산유스호텔-', '#숙소다녀왔습니다_제천📍충북']",월악산유스호텔,1
1,🏷 몸과 마음을 편히 기대어 쉴수 있는 #누울- - - - - - - - - - -...,2021-09-05,85,Jeju,"['#누울-', '#숙소다녀왔습니다_제주📍제주시']",누울,1
2,🏷 한옥독채 휴양 풀빌라 #빌라오- - - - - - - - - - - - - - ...,2021-09-03,85,순천,"['#빌라오-', '#숙소다녀왔습니다_순천📍순천']",빌라오,1
3,🏷 공공연한 비밀 #공공연- - - - - - - - - - - - - - - - ...,2021-09-01,82,Jeju,"['#공공연-', '#숙소다녀왔습니다_제주📍제주시']",공공연,1
4,🏷 프리미엄 해양 리조트 #벨버디어- - - - - - - - - - - - - -...,2021-08-31,77,거제벨버디어,"['#벨버디어-', '#숙소다녀왔습니다_거제📍경남']",벨버디어,1
...,...,...,...,...,...,...,...
101,🏷한옥감성 분위기끝판왕 #서유숙펜션 ⠀- - - - - - - - - - - - -...,2021-03-09,28,서유숙펜션,"['#서유숙펜션', '#충북', '#충주', '#서유슉펜션📍충북']",서유숙펜션,1
102,🏷산속 프라이빗한 온수풀 풀빌라#헤리티스⠀- - - - - - - - - - - -...,2021-03-06,27,헤리티스풀빌라,"['#헤리티스⠀-', '#강원도', '#홍천', '#헤리티스풀빌라📍강원']",헤리티스,1
103,🏷포레스트뷰 감상하며 힐링하기 좋은숙소#무무펜션⠀- - - - - - - - - -...,2021-03-05,36,무무펜션,"['#무무펜션⠀-', '#인천', '#강화도', '#무무펜션📍인천']",무무펜션,1
104,🏷노천탕이 매력적인 가덕도 펜션 #갤러리가덕⠀- - - - - - - - - - -...,2021-03-03,24,None,"['#갤러리가덕⠀-', '#부산', '#갤러리가덕📍부산']",갤러리가덕,1


In [527]:


def convert(x):
    return int(x.replace(',',''))

if rest_behappyhere.like.dtype == 'O':
    rest_behappyhere.like = rest_behappyhere.like.apply(lambda x : convert(x))


# 1, 중복된 이름 검색 
# 2.개수가 1인거 이상 뽑아내기 
# 3.리스트만들기
rb_cnt = Counter(rest_behappyhere.name)      # Counter 모듈로 name data count
li =[]
for name, cnt  in rb_cnt.items():            # 중복된 것만 뽑기 (cnt가 1이상이면 중복)
    if cnt > 1:
        li.append(name)
         

# 4. 행가져와서 like집계, content, tag list append
for i in li:           # 중복된 name 반복
    ex = rest_behappyhere.loc[rest_behappyhere.name == i]  # 반복된 name의 행을 가져온다
    ex.reset_index(drop=True,inplace=True)     # index reset
    
    content = ex.iloc[0,:].content
    date = ex.iloc[0,:].date
    like = ex.iloc[0,:].like
    place = ex.iloc[0,:].place
    tags = ex.iloc[0,:].tags
    name = ex.name[0]
    
    for i in range(len(ex)-1):
        content += ex.iloc[i+1,:].content
        like += ex.iloc[i+1,:].like
        tags += ex.iloc[i+1,:].tags
        
    
    
    # 5.차례대로 기존 행 지워내기
    indexNames = rest_behappyhere[rest_behappyhere['name'] == ex.name[0]].index
    rest_behappyhere.drop(indexNames, inplace=True) 
    rest_behappyhere = rest_behappyhere.append(pd.Series(data=[content,date,like,place,tags,name],index=rest_behappyhere.columns),ignore_index=True)

rb_cnt_tmp = []
for i in rest_behappyhere.name:
    rb_cnt_tmp.append(rb_cnt[i])

rest_behappyhere['overlap'] = rb_cnt_tmp
rest_behappyhere

,content,date,like,place,tags,name,overlap
0,그림같은 일몰과 다채로운 자연의 색깔을 즐길 수 있는 곳이것이야말로 가을 힐링✨ #...,2021-09-06,183,None,"['#서산', '#제로플레이스', '#숙소기록', '#숙소기록_국내', '#숙소기록...",제로플레이스,1
1,⠀시골집에 온 듯 편안한 분위기에서노천탕을 즐길 수 있는 #청도 #루오스테이 🛁⠀대...,2021-09-06,608,경북 청도,"['#청도', '#루오스테이', '#숙소기록', '#숙소기록_국내', '#숙소기록_...",루오스테이,1
2,⠀한적하고 조용하게 쉬어가며 #힐링할 수있는 #그루브스테이✨✨⠀예쁜 숙소에서 조용히...,2021-09-03,345,None,"['#그루브스테이✨✨⠀예쁜', '#숙소기록', '#숙소기록_국내', '#숙소기록_부...",그루브스테이,1
3,높은 가을하늘 바라보며 힐링할 수 있는💙#노천탕숙소 모음 💙-📍 #강릉 #목가블루목...,2021-09-02,308,None,"['#노천탕숙소', '#강릉', '#목가블루목조한옥', '#남해', '#적정온도노을...",강릉,1
4,서해바다의 아름다은 일몰 맛집 캠핑장🏕#태안 #사목공원캠핑장 캠핑장 대부분의 사이트...,2021-09-01,405,None,"['#태안', '#사목공원캠핑장', '#태안사목공원캠핑장', '#충남캠핑장', '#...",사목공원캠핑장,1
...,...,...,...,...,...,...,...
86,⠀트로피컬한 인피니티풀에서 힐링할 수 있는 #제주 #트로피컬하이드어웨이 🌊⠀-⠀절반...,2021-07-07,42,None,"['#제주', '#트로피컬하이드어웨이', '#숙소기록', '#숙소기록_국내', '#...",트로피컬하이드어웨이,1
87,"⠀하와이 부럽지 않은 오션뷰 인피니티 풀 빌라, #여수 #블루망고풀빌라 🌊⠀-⠀야자...",2021-07-07,40,None,"['#여수', '#블루망고풀빌라', '#숙소기록', '#숙소기록_국내', '#숙소기...",블루망고풀빌라,1
88,⠀몰디브에서 투명카약 어때? #통영 #드레피인 🛶⠀-⠀보기만 해도 청량함이 느껴지...,2021-07-06,49,None,"['#통영', '#드레피인', '#숙소기록', '#숙소기록_국내', '#숙소기록_통영']",드레피인,1
89,"⠀⠀지역별 노천탕 숙소모음✨✨@@가을에 자기랑 갈 곳 미리 메모,,,끄적X2⠀⠀-⠀...",2021-09-06,259,None,"['#숙소기록', '#숙소기록_국내⠀⠀⠀', '#숙소', '#숙소추천', '#국내숙...",숙소기록,1


In [529]:


def convert(x):
    return int(x.replace(',',''))

if sookso_diary.like.dtype == 'O':
    sookso_diary.like = sookso_diary.like.apply(lambda x : convert(x))


# 1, 중복된 이름 검색 
# 2.개수가 1인거 이상 뽑아내기 
# 3.리스트만들기
sd_cnt = Counter(sookso_diary.name)      # Counter 모듈로 name data count
li =[]
for name, cnt  in sd_cnt.items():            # 중복된 것만 뽑기 (cnt가 1이상이면 중복)
    if cnt > 1:
        li.append(name)

# 4. 행가져와서 like집계, content, tag list append
for i in li:           # 중복된 name 반복
    ex = sookso_diary.loc[sookso_diary.name == i]  # 반복된 name의 행을 가져온다
    ex.reset_index(drop=True,inplace=True)     # index reset
    
    content = ex.iloc[0,:].content
    date = ex.iloc[0,:].date
    like = ex.iloc[0,:].like
    place = ex.iloc[0,:].place
    tags = ex.iloc[0,:].tags
    name = ex.name[0]
    
    for i in range(len(ex)-1):
        content += ex.iloc[i+1,:].content
        like += ex.iloc[i+1,:].like
        tags += ex.iloc[i+1,:].tags
        
    
    
    # 5.차례대로 기존 행 지워내기
    indexNames = sookso_diary[sookso_diary['name'] == ex.name[0]].index
    sookso_diary.drop(indexNames, inplace=True) 
    sookso_diary = sookso_diary.append(pd.Series(data=[content,date,like,place,tags,name],index=sookso_diary.columns),ignore_index=True)

sd_cnt_tmp = []
for i in sookso_diary.name:
    sd_cnt_tmp.append(sd_cnt[i])

sookso_diary['overlap'] = sd_cnt_tmp
sookso_diary

,content,date,like,place,tags,name,overlap
0,"황리단길과 가까운 한옥 풀빌라 독채숙소 ""경주_담안""경주만의 고즈넉한 느낌이 가득한...",2021-09-06,100,None,"['#경주', '#여행', '#경주숙소', '#경주여행', '#담안', '#경주담안...",담안,1
1,"애견동반 가능한 독채 전원 펜션, ""인제_메종카누네""초록초록한 넓은 정원에 반려견과...",2021-09-05,54,None,"['#강원도', '#인제', '#강원도펜션', '#인제펜션', '#메종카누네', '...",메종카누네,1
2,✨평일 숙박권 이벤트✨숙소일기 팔로워+게시글 좋아요+친구태그 하시면2명 댓글추첨을 ...,2021-09-04,77,None,"['#영덕', '#느와펜션', '#느와&블르', '#숙소', '#감성숙소', '#숙...",느와,1
3,"여수에 위치한 독채 자쿠지 펜션 ""하월""세심한 인테리어와 소품들이 완죠니 감성적이야...",2021-09-03,76,여수,"['#여수', '#여수여행', '#여수숙소', '#하월', '#감성숙소', '#여수...",하월,1
4,🎉평일 숙박권 이벤트🎉숙소일기 팔로워+게시글 좋아요+친구태그 하시면1명 댓글추첨을 ...,2021-09-03,260,None,"['#강원도', '#양양', '#스테이해연', '#양양스테이해연', '#감성숙소',...",스테이해연,1
...,...,...,...,...,...,...,...
231,.비대면으로 체크인하는 경주 독채한옥🏡개별독채로 이루어져 프라이빗한 공간!싸장님의 ...,2021-03-04,129,None,"['#경주', '#소이한옥', '#황리단길숙소', '#경주한옥숙소', '#경주숙소'...",소이한옥,1
232,부산에서 노을질 때 가장 예쁘다는 #광안리호텔1광안리 바다와 광안대교의 환상적인 조...,2021-03-03,80,None,"['#광안리호텔1광안리', '#광안리', '#광안리HOTEL1', '#광안리호텔',...",광안리,1
233,"음악이 있는 햇살맛집숙소! 제주도에 위치한 ""춤추는달""🌙감귤밭과 어울어진 아기자기한...",2021-08-26,109,None,"['#제주', '#제주도', '#제주도여행', '#제죽숙소', '#춤추는달', '#...",춤추는달,2
234,🎉평일 숙박권 이벤트🎉숙소일기 팔로워+게시글 좋아요+친구태그 하시면1명 댓글추첨을 ...,2021-07-30,307,None,"['#경주', '#여행', '#경주숙소', '#읍성별장', '#경주여행', '#감성...",읍성별장,2


---

In [530]:
'''
1. 중복된 이름 검색
2. 개수가 1인거 이상 뽑아내기
3. 리스트만들기
4. 행가져와서 like집계, content, tag 리스트로 append
5. 차례대로 기존 행 지워내기
6. place 네이버 API 사용해서 대체
'''



# like integet 변환 후 합치기
def convert(x):
    return int(x.replace(',',''))

if sookso_hada.like.dtype == 'O':
    sookso_hada.like = sookso_hada.like.apply(lambda x : convert(x))




# 1, 중복된 이름 검색 
# 2.개수가 1인거 이상 뽑아내기 
# 3.리스트만들기
sh_cnt = Counter(sookso_hada.name)      # Counter 모듈로 name data count
li =[]
for name, cnt  in sh_cnt.items():            # 중복된 것만 뽑기 (cnt가 1이상이면 중복)
    if cnt > 1:
        li.append(name)        
        

# 4. 행가져와서 like집계, content, tag list append
for i in li:           # 중복된 name 반복
    ex = sookso_hada.loc[sookso_hada.name == i]  # 반복된 name의 행을 가져온다
    ex.reset_index(drop=True,inplace=True)     # index reset
    
    content = ex.iloc[0,:].content
    date = ex.iloc[0,:].date
    like = ex.iloc[0,:].like
    place = ex.iloc[0,:].place
    tags = ex.iloc[0,:].tags
    name = ex.name[0]
    
    for i in range(len(ex)-1):
        content += ex.iloc[i+1,:].content
        like += ex.iloc[i+1,:].like
        tags += ex.iloc[i+1,:].tags
    
    # 5.차례대로 기존 행 지워내기
    indexNames = sookso_hada[sookso_hada['name'] == ex.name[0]].index
    sookso_hada.drop(indexNames, inplace=True) 
    sookso_hada = sookso_hada.append(pd.Series(data=[content,date,like,place,tags,name],index=sookso_hada.columns),ignore_index=True)

sh_cnt_tmp = []
for i in sookso_hada.name:
    sh_cnt_tmp.append(sh_cnt[i])

sookso_hada['overlap'] = sh_cnt_tmp
sookso_hada

,content,date,like,place,tags,name,overlap
0,"독서의 계절, 가을 !책을 벗삼아 머무르기 좋은 #북스테이 추천 BEST 51. #...",2021-09-06,16,None,"['#북스테이', '#품안의숲', '#썸원스페이지숲', '#이후북스테이', '#갖가...",품안의숲,1
1,"#완주 #느린집 반려견동반이 가능한 완주 숙소, 느린집입니다.느린집은 완주의 어느 ...",2021-09-04,17,완주군,"['#완주', '#느린집', '#반려견동반펜션', '#파티룸', '#숙소하다_완주'...",느린집,1
2,"#제천 #리솜포레스트 자연과 교감하며 일상의 피로를 회복할 수 있는 곳, 제천에 위...",2021-09-03,30,리솜 포레스트 Resom Forest,"['#제천', '#리솜포레스트', '#숲뷰', '#프라이빗스파', '#숙소하다_제천...",리솜포레스트,1
3,"#양산 #스테이무메 휴식하며 오순도순 이야기하기 좋은 부산근교 양산에 위치한 숙소,...",2021-09-02,26,양산,"['#양산', '#스테이무메', '#부산근교여행', '#양산숙소', '#숙소하다_양...",스테이무메,1
4,"#홍천 #올라운드원 자연과 어우러져 있는 홍천의 원형 독채펜션, 올라운드원입니다. ...",2021-09-01,23,올라운드원,"['#홍천', '#올라운드원', '#홍천독채펜션', '#홍천숙소', '#숙소하다_홍...",올라운드원,1
...,...,...,...,...,...,...,...
101,#거제도 #디스커버리풀빌라 거제도의 #뷰맛집 펜션으로 유명한 디스커버리풀빌라입니다....,2021-05-03,7,거제도,"['#거제도', '#디스커버리풀빌라', '#뷰맛집', '#거가대교', '#이수도',...",디스커버리풀빌라,1
102,"#부산 #청학소담 부산 영도에 위치한 감성적인 옛날 주택, 청학소담입니다. 청학소담...",2021-05-02,7,부산,"['#부산', '#청학소담', '#부산항대교', '#부산감성숙소', '#부산바다',...",청학소담,1
103,#강릉 #느린가 느린가(구 #하슬라스테이 )는 강릉 교동에 위치한 한옥 스테이입니다...,2021-05-01,8,None,"['#강릉', '#느린가', '#하슬라스테이', '#인생샷', '#한옥스테이', '...",느린가,1
104,#제주도 #클랭블루스테이 제주 뷰맛집 카페로 유명한 클랭블루에서 이번에 새로 오픈한...,2021-04-30,13,제주도,"['#제주도', '#클랭블루스테이', '#제주숙소', '#제주일몰', '#제주인생샷...",클랭블루스테이,1


---

##  데이터 프레임 합치기

In [531]:
tot_dataset = daily_gamsung
for df in [gamsung_bnb, gamsung_curation, hi_stay_tour, rest_behappyhere, sookso_diary, sookso_hada]:
    tot_dataset = tot_dataset.append(df)

In [532]:
tot_dataset

,content,date,like,place,tags,name,overlap
0,"속초_파도의무늬오션뷰 감성 에어비앤비""파도의무늬""'파도의무늬'는물결이 일렁이고 윤슬...",2021-09-06,39,파도의 무늬,"['#데일리감성속초로', '#데일리감성_속초강원도', '#데일리감성_강원——————...",파도의무늬,1
1,"거제_큐브스테이캠핑 감성의 펜션""큐브스테이""'큐브스테이'는거제도 아름다운 섬 속에두...",2021-09-04,51,Geoje,"['#데일리감성거제로', '#데일리감성_거제경상남도', '#데일리감성_경남—————...",큐브스테이,1
2,"경주_조히경주 독채 한옥 감성숙소""조히""'조히'는ㄱ자로 되어 있는 독채 한옥 숙소로...",2021-09-03,43,경주 황리단길,"['#데일리감성경주로', '#데일리감성_경주경상북도', '#데일리감성_경북—————...",조히,1
3,"강릉_스테이 시호일시간의 흔적을 담은 감성숙소""스테이 시호일""'스테이 시호일'은60...",2021-09-03,12,강릉,"['#데일리감성강릉으로', '#데일리감성_강릉강원도', '#데일리감성_강원—————...",스테이 시호일,1
4,"제주_까만돌멩이구좌읍 조식주는 예쁜 숙소""까만돌멩이""'까만돌멩이'는가족 및 커플 또...",2021-09-03,33,제주 까만돌멩이,"['#데일리감성제주도', '#데일리감성_제주—————————————————']",까만돌멩이,1
...,...,...,...,...,...,...,...
101,#거제도 #디스커버리풀빌라 거제도의 #뷰맛집 펜션으로 유명한 디스커버리풀빌라입니다....,2021-05-03,7,거제도,"['#거제도', '#디스커버리풀빌라', '#뷰맛집', '#거가대교', '#이수도',...",디스커버리풀빌라,1
102,"#부산 #청학소담 부산 영도에 위치한 감성적인 옛날 주택, 청학소담입니다. 청학소담...",2021-05-02,7,부산,"['#부산', '#청학소담', '#부산항대교', '#부산감성숙소', '#부산바다',...",청학소담,1
103,#강릉 #느린가 느린가(구 #하슬라스테이 )는 강릉 교동에 위치한 한옥 스테이입니다...,2021-05-01,8,None,"['#강릉', '#느린가', '#하슬라스테이', '#인생샷', '#한옥스테이', '...",느린가,1
104,#제주도 #클랭블루스테이 제주 뷰맛집 카페로 유명한 클랭블루에서 이번에 새로 오픈한...,2021-04-30,13,제주도,"['#제주도', '#클랭블루스테이', '#제주숙소', '#제주일몰', '#제주인생샷...",클랭블루스테이,1


In [533]:
len(tot_dataset.name.unique())

625

In [ ]:
def convert(x):
    return int(x.replace(',',''))

if tot_dataset.like.dtype == 'O':
    tot_dataset.like = tot_dataset.like.apply(lambda x : convert(x))
    
    
    
dg_cnt = Counter(tot_dataset.name)      # Counter 모듈로 name data count
li =[]
for name, cnt  in dg_cnt.items():            # 중복된 것만 뽑기 (cnt가 1이상이면 중복)
    if cnt > 1:
        li.append(name)
            

# 4. 행가져와서 like집계, content, tag list append
for i in li:           # 중복된 name 반복
    ex = tot_dataset.loc[tot_dataset.name == i]  # 반복된 name의 행을 가져온다
    ex.reset_index(drop=True,inplace=True)     # index reset
    
    content = ex.iloc[0,:].content
    date = ex.iloc[0,:].date
    like = ex.iloc[0,:].like
    place = ex.iloc[0,:].place
    tags = ex.iloc[0,:].tags
    name = ex.name[0]
    overlap = ex.iloc[0,:].overlap
    
    for i in range(len(ex)-1):
        content += ex.iloc[i+1,:].content
        like += ex.iloc[i+1,:].like
        tags += ex.iloc[i+1,:].tags
        overlap += ex.iloc[i+1,:].overlap
    
    
    # 5.차례대로 기존 행 지워내기
    indexNames = tot_dataset[tot_dataset['name'] == ex.name[0]].index
    tot_dataset.drop(indexNames, inplace=True) 
    tot_dataset = tot_dataset.append(pd.Series(data=[content,date,like,place,tags,name,overlap],index=tot_dataset.columns),ignore_index=True)

In [543]:
tot_dataset[tot_dataset.name == '온화한가']

,content,date,like,tags,name,overlap,place
398,#청도_온화한가경북 청도에 위치하고있는 조용하고 평화로운 독채풀빌라숙소 '온화한가'...,2021-08-08,1398,"['#청도_온화한가경북', '#감성비앤비🏷청도로', '#감성비앤비_청도사진출처:',...",온화한가,2,


---

## Naver 장소 API

In [537]:
# naver application clientID,secret
client_id = "6E6mBjvxbdEgVk6Prqgd"
client_secret = "0ZjWnYiBjr"


# quote가 인코딩을 변경(UTF8로 url에 넘겨주기 위해서)
# 내가 입력한 값을 기준으로 parse
title_et = []
for name in tot_dataset.name:
    query = urllib.parse.quote(name)
    idx = 0 # 요청 개수
    display = 10 # 100개 단위로 가져온다
    start = 1
    end = 10
    tmp=''
    
    url = "https://openapi.naver.com/v1/search/local?query=" + query \
        + "&display=" + str(display) \
        + "&start=" + str(start)


    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    # request 요청에 대한 결과를 response에 저장
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
        response_dict = json.loads(response_body.decode('utf-8'))
        items = response_dict['items']
        for item_index in range(0,len(items)):
            remove_tag = re.compile('<.*?>')
            title = re.sub(remove_tag, '', items[item_index]['title'])
            link = items[item_index]['link']
            caterory = items[item_index]['category']
            description = re.sub(remove_tag, '',items[item_index]['description'])
            telephone = items[item_index]['telephone']
            address = items[item_index]['address']
            roadAddress = items[item_index]['roadAddress']
            mapx = items[item_index]['mapx']
            mapy = items[item_index]['mapy']

            # 이름이 없는 경우
            if not title : 
                title_et.append('')
            else:    
                if caterory[:2] == '숙박' :  # 카테고리가 숙박인 경우
                    if name in title and title in name :
                        tmp = roadAddress
                        
        if tmp : title_et.append(tmp)
        else: title_et.append('')

    else:
        print("Error Code:" + rescode)

    time.sleep(0.1)


In [538]:
tot_dataset.drop('place',axis=1,inplace=True)

In [539]:
tot_dataset['place'] = title_et

In [540]:
tot_dataset

,content,date,like,tags,name,overlap,place
0,"속초_파도의무늬오션뷰 감성 에어비앤비""파도의무늬""'파도의무늬'는물결이 일렁이고 윤슬...",2021-09-06,39,"['#데일리감성속초로', '#데일리감성_속초강원도', '#데일리감성_강원——————...",파도의무늬,1,
1,"강릉_스테이 시호일시간의 흔적을 담은 감성숙소""스테이 시호일""'스테이 시호일'은60...",2021-09-03,12,"['#데일리감성강릉으로', '#데일리감성_강릉강원도', '#데일리감성_강원—————...",스테이 시호일,1,
2,"제주_까만돌멩이구좌읍 조식주는 예쁜 숙소""까만돌멩이""'까만돌멩이'는가족 및 커플 또...",2021-09-03,33,"['#데일리감성제주도', '#데일리감성_제주—————————————————']",까만돌멩이,1,제주특별자치도 제주시 구좌읍 계룡길 32
3,"단양_권장사양인스타감성 가득한 독채 숙소""권장사양""'권장사양'은 자연경관으로 둘러쌓...",2021-09-02,36,"['#데일리감성단양으로', '#데일리감성_단양충청북도', '#데일리감성_충북————...",권장사양,1,
4,"강릉_라온가따뜻한 느낌의 힐링 감성숙소""라온가""'라온가'는 주택을 개조한 숙소이며'...",2021-09-01,24,"['#데일리감성강릉으로', '#데일리감성_강릉강원도', '#데일리감성_강원—————...",라온가,1,
...,...,...,...,...,...,...,...
604,".한 폭의 그림같은 오션뷰, ""제주_Salt”🌊✨액자 창문 너머로 보이는 바다만 보...",2021-06-24,142,"['#제주', '#솔트', '#제주숙소', '#제주여행', '#제주숙소추천', '#...",솔트,2,
605,".통창 리버뷰가 환상적인 ""춘천_피그멜리온이펙트""리버+마운틴뷰를 보며 월풀스파를 즐...",2021-06-15,87,"['#춘천', '#피그멜리온이펙트', '#춘천숙소', '#춘천여행', '#춘천숙소추...",피그멜리온이펙트,2,강원도 춘천시 서면 경춘로 405
606,"작은 시골 동네 안에 위치한 오붓&편안한 감성숙소,곡성_품안의밤!🧡논밭뷰를 보며 여...",2021-04-21,68,"['#곡성', '#품안의밤', '#곡성숙소', '#곡성여행', '#곡성숙소추천', ...",품안의밤,2,
607,"KBS2에서 방송됐던 ""한 번 다녀왔습니다""의 웨딩촬영지로 유명한 강릉_더브리앙뜨!...",2021-04-19,57,"['#강릉', '#더브리앙뜨', '#강릉숙소', '#강릉여행', '#강릉데이트', ...",더브리앙뜨,2,강원도 강릉시 사천면 중앙서로 92-43


In [544]:
tot_dataset.to_csv('tot_dataset.csv')